# Tensorflow and Computer Vision

TensorFlow is one of many deep learning frameworks available to researchers and developers to enhance their applications with machine learning. AWS provides broad support for TensorFlow, enabling customers to develop and serve their own models across computer vision, natural language processing, speech translation, and more.

You can get started with TensorFlow on AWS using Amazon SageMaker, a fully managed service that provides machine learning (ML) developers and data scientists with the ability to build, train, and deploy ML models quickly.

### Background:

You are tasked to create a Machine Learning Model which is capable of identifying a cat or dog or frog that visits the city vegetable garden. You decided to use the TensorFlow framework to solve object identification problems. You are using the CIFAR-10 dataset as a sample dataset to investigate your approach.

### About the dataset
The CIFAR-10 dataset consists of 60,000 32x32 color images in 10 classes, with 6000 images per class. There are 50,000 training images and 10,000 test images:

![](img/cifar-10.png)


This lab is designed based on the aws-samples available in the following [github repo](https://github.com/aws-samples/sagemaker-multi-model-endpoint-tensorflow-computer-vision).

**Note** To run the code cell, press **SHIFT + ENTER**. When the cell finishes running, the text to the left of the cell changes from **In [*]:** to **In [1]**.

## Setup

 #### Prerequisites 
 Update the Sagemaker to the latest version and install the TensorFlow framework 2.6.1.

#### Code Cell 1

In [ ]:
%%capture
%pip install sagemaker
%pip install tensorflow
%pip install matplotlib

### Code Cell 2

In [ ]:
# Validate if the TensorFlow is installed correctly.
import tensorflow as tf
print(f"TensorFlow version:{tf.__version__}")

#### Imports 

### Code Cell 3

In [ ]:
import os
import time
import boto3
import logging
import sagemaker
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras import utils
from sagemaker.tensorflow import TensorFlow
from sagemaker.inputs import TrainingInput
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sagemaker.tensorflow.serving import TensorFlowModel
from tensorflow.keras.datasets import cifar10

from sagemaker import get_execution_role

#### Setup Logger

### Code Cell 4

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

### Code Cell 5

In [ ]:
logger.info(f'[Using TensorFlow version: {tf.__version__}]')
logger.info(f'[Using SageMaker version: {sagemaker.__version__}]')

#### Seed for Reproducibility

### Code Cell 6

In [ ]:
SEED = 123
np.random.seed(SEED)
tf.random.set_seed(SEED)

#### Create Roles, Sessions and Data Locations

### Code Cell 7

In [ ]:
role = get_execution_role()
session = boto3.Session()
sagemaker_session = sagemaker.Session()

s3 = session.resource('s3')
TF_FRAMEWORK_VERSION = '2.11.0'
BUCKET = '<Enter_Your_Bucket_Name>'
PREFIX = 'cv-models'

## Train Model - CIFAR-10 Image Classification

<p align="justify">First, we will train a Convolutional Neural Network (CNN) model to classify images from the CIFAR-10 dataset. Image classification is the task of assigning a label to an image, from a predefined set of categories. CIFAR-10 is an established CV dataset used for object recognition. It is a subset of the 80 Million Tiny Images dataset and consists of 60,000 (32x32) color images containing 1 of 10 object classes, with 6,000 images per class.</p>

#### a) Load Data

The first step is to load the pre-shuffled CIFAR-10 dataset into our train and test objects. Luckily, Keras provides the CIFAR dataset for us to load using the `load_data()` method. All we have to do is import keras.datasets and then load the data.

### Code Cell 8

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

### Code Cell 9

In [ ]:
logger.info(f'X_train Shape: {X_train.shape}')
logger.info(f'y_train Shape: {y_train.shape}')
logger.info(f'X_test Shape : {X_test.shape}')
logger.info(f'y_test Shape : {y_test.shape}')

#### b) Data Exploration

Let's review the subset of data.

### Code Cell 10

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(20, 5))
for i in range(36):
    ax = fig.add_subplot(3, 12, i+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(X_train[i]))

#### c) Data Preparation

##### Rescale 
Rescales the images by dividing the pixel values by 255: [0,255] ⇒ [0,1]

### Code Cell 11

In [ ]:
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

##### One Hot Encode Target Labels
<p align="justify">One-hot encoding is a process by which categorical variables are converted into a numeric form. One-hot encoding converts the (1 × n) label vector to a label matrix of dimensions (10 × n), where n is the number of sample images. So, if we have 1,000 images in our dataset, the label vector will have the dimensions (1 × 1000). After one-hot encoding, the label matrix dimensions will be (1000 × 10). That’s why, when we define our network architecture in the next step, we will make the output softmax layer contain 10 nodes, where each node represents the probability of each class we have.</p>

### Code Cell 12

In [ ]:
num_classes = len(np.unique(y_train))
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

##### Split Data
Break the original train data set further into train and validation data sets.

### Code Cell 13

In [ ]:
X_train, X_validation = X_train[500:], X_train[:500]
y_train, y_validation = y_train[500:], y_train[:500]

##### Save to Local

Create a local `data/cifar_10` directory to save the datasets.

### Code Cell 14

In [ ]:
DATASET_PATH = './data/cifar_10'

### Code Cell 15

In [ ]:
os.makedirs(DATASET_PATH, exist_ok=True)

Save train, validation and test sets to local `data` directory

### Code Cell 16

In [ ]:
np.save(f'{DATASET_PATH}/X_train.npy', X_train)
np.save(f'{DATASET_PATH}/y_train.npy', y_train)
np.save(f'{DATASET_PATH}/X_validation.npy', X_validation)
np.save(f'{DATASET_PATH}/y_validation.npy', y_validation)
np.save(f'{DATASET_PATH}/X_test.npy', X_test)
np.save(f'{DATASET_PATH}/y_test.npy', y_test)

##### Copy Datasets to S3
Copy train, validation and test sets from the local dir to S3, since SageMaker expects datasets to be in S3 for training.

### Code Cell 17

In [ ]:
!aws s3 cp ./{DATASET_PATH}/X_train.npy s3://{BUCKET}/{PREFIX}/cifar_10/train/
!aws s3 cp ./{DATASET_PATH}/y_train.npy s3://{BUCKET}/{PREFIX}/cifar_10/train/
!aws s3 cp ./{DATASET_PATH}/X_validation.npy s3://{BUCKET}/{PREFIX}/cifar_10/validation/
!aws s3 cp ./{DATASET_PATH}/y_validation.npy s3://{BUCKET}/{PREFIX}/cifar_10/validation/
!aws s3 cp ./{DATASET_PATH}/X_test.npy s3://{BUCKET}/{PREFIX}/cifar_10/test/
!aws s3 cp ./{DATASET_PATH}/y_test.npy s3://{BUCKET}/{PREFIX}/cifar_10/test/

#### d) Create Training Inputs 
Using the S3 locations of the datasets we saved in the previous step, create pointers to these datasets using the `TrainingInput`class from the SageMaker SDK.

### Code Cell 18

In [ ]:
train_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/cifar_10/train', 
                            distribution='FullyReplicated', 
                            content_type='npy')
validation_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/cifar_10/validation', 
                                 distribution='FullyReplicated', 
                                 content_type='npy')
test_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/cifar_10/test', 
                           distribution='FullyReplicated', 
                           content_type='npy')

### Code Cell 19

In [ ]:
inputs = {'train': train_input, 'val': validation_input, 'test': test_input}

#### e) Define Model Architecture & create Training Script

We will build a small CNN consisting of three convolutional layers and two dense layers.<br>
<b>Note:</b> We will use the ReLU activation function for all the hidden layers. In the last dense layer, we will use a softmax activation function with 10 nodes to return an array of 10 probability scores (summing to 1). Each score will be the probability that the current image belongs to our 10 image classes.

### Code Cell 20

In [ ]:
!pygmentize cifar_train.py

#### f) Create a TensorFlow Estimator & fit the Model

### Code Cell 21

In [ ]:
model_name = 'cifar-10'
instance_type = 'ml.m5.xlarge'
hyperparameters = {'epochs': 12}

estimator_parameters = {'entry_point':'cifar_train.py',
                        'instance_type': instance_type,
                        'instance_count': 1,
                        'model_dir': f'/opt/ml/model',
                        'role': role,
                        'hyperparameters': hyperparameters,
                        'output_path': f's3://{BUCKET}/{PREFIX}/cifar_10/out',
                        'base_job_name': f'cv-{model_name}',
                        'framework_version': TF_FRAMEWORK_VERSION,
                        'py_version': 'py39',
                        'script_mode': True}
estimator = TensorFlow(**estimator_parameters)

### Code Cell 22

In [ ]:
estimator.fit(inputs)

## Create an Endpoint

#### a) Copy Trained Models to a common S3 Prefix

### Code Cell 23

In [ ]:
tf_model = estimator.model_data
output = f's3://{BUCKET}/{PREFIX}/cifar.tar.gz'

### Code Cell 24

In [ ]:
!aws s3 cp {tf_model} {output}

#### b) Essentials

### Code Cell 25

In [ ]:
current_time = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d-%H-%M-%S')
current_time

### Code Cell 26

In [ ]:
IMAGE_URI = '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.6-gpu-py38-cu112-ubuntu20.04-v1'
model_data_prefix = f's3://{BUCKET}/{PREFIX}/'

#### c) Define a Tensor Flow Model

### Code Cell 27

In [ ]:
model = TensorFlowModel(model_data=output, 
                          role=role, 
                          image_uri=IMAGE_URI)

#### d) Deploy the model to an Endpoint

### Code Cell 28

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer


instance_type = "ml.m5.xlarge"

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
)

## Test the Endpoint for Real Time Inference 

#### a) Test CIFAR Image Classification

### Code Cell 29

In [ ]:
%matplotlib inline
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing import image
from IPython.display import Image
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt
import numpy as np

### Code Cell 30

In [ ]:
CIFAR10_LABELS = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

### Code Cell 31

In [ ]:
Image('./data/cifar_10/raw_images/cat.png')

### Code Cell 32

In [ ]:
img = load_img('./data/cifar_10/raw_images/cat.png', target_size=(32, 32))
data = img_to_array(img)
data = data.astype('float32')
data = data / 255.0
data = data.reshape(1, 32, 32, 3)

### Code Cell 33

In [ ]:
payload = {'instances': data}

### Code Cell 34

In [ ]:
y_pred = predictor.predict(data=payload)

### Code Cell 35

In [ ]:
np.argmax(y_pred['predictions'])

### Code Cell 36

In [ ]:
CIFAR10_LABELS[np.argmax(y_pred['predictions'])]

## Cleanup (Optional)

Uncomment the below code block to delete the Sagemaker Endpoint.

### Code Cell 37

In [ ]:
#predictor.delete_endpoint()